In [ ]:
import os
import json
from models.vehicle import Car, Truck, Bike

class FleetManager:
    def __init__(self):
        self.vehicles = []

    def add_vehicle(self, vehicle):
        self.vehicles.append(vehicle)

    def rent_vehicle(self, vehicle_name):
        # Transportni topish
        vehicle = self.find_vehicle(vehicle_name)
        if vehicle and vehicle.available:
            vehicle.rent()
            self.log_action(f"{vehicle.name} rented for ${vehicle.calculate_rent():.2f}")
        else:
            print(f"Transport {vehicle_name} mavjud emas yoki ijaraga berilmagan.")

    def return_vehicle(self, vehicle_name):
        # Transportni qaytarish
        vehicle = self.find_vehicle(vehicle_name)
        if vehicle and not vehicle.available:
            vehicle.return_vehicle()
            self.log_action(f"{vehicle.name} returned.")
        else:
            print(f"Transport {vehicle_name} hali ijaraga olinmagan.")

    def find_vehicle(self, vehicle_name):
        # Transportni nomi bo‘yicha izlash
        for vehicle in self.vehicles:
            if vehicle.name.lower() == vehicle_name.lower():
                return vehicle
        return None

    def search(self, **criteria):
        # Transportlarni qidirish
        results = []
        for v in self.vehicles:
            match = True
            for key, value in criteria.items():
                if str(getattr(v, key)).lower() != str(value).lower():
                    match = False
                    break
            if match:
                results.append(v)
        return results

    def save_to_json(self, path='data/vehicles.json'):
        # Transportlarni JSON faylga saqlash
        os.makedirs(os.path.dirname(path), exist_ok=True)
        vehicles_data = [v.to_dict() for v in self.vehicles]
        try:
            with open(path, 'w') as f:
                json.dump(vehicles_data, f, indent=4)
        except Exception as e:
            print(f"Error while saving data: {e}")

    def load_from_json(self, path='data/vehicles.json'):
        # JSON faylini yuklab olish
        if not os.path.exists(path):
            print("JSON fayli topilmadi. Yangi fayl yaratiladi.")
            return

        try:
            with open(path, 'r') as f:
                vehicles_data = json.load(f)
                if vehicles_data:
                    for v_data in vehicles_data:
                        if v_data['type'] == 'car':
                            self.add_vehicle(Car(v_data['name'], v_data['price']))
                        elif v_data['type'] == 'truck':
                            self.add_vehicle(Truck(v_data['name'], v_data['price']))
                        elif v_data['type'] == 'bike':
                            self.add_vehicle(Bike(v_data['name'], v_data['price']))
        except json.JSONDecodeError:
            print("JSON format xatosi. Iltimos faylni tekshirib ko‘ring.")

    def log_action(self, action):
        # Harakatni log qilish
        print(action)


In [ ]:
from abc import ABC, abstractmethod

class Vehicle(ABC):
    def __init__(self, name, price, vtype):
        self.name = name
        self.price = price
        self.available = True
        self.rent_count = 0
        self.type = vtype  # Yangi atribut qo‘shildi

    @abstractmethod
    def rent(self):
        pass

    @abstractmethod
    def return_vehicle(self):
        pass

    @abstractmethod
    def calculate_rent(self):
        pass

    def to_dict(self):
        return {
            "name": self.name,
            "price": self.price,
            "type": self.type,
            "available": self.available,
            "rent_count": self.rent_count
        }

class Car(Vehicle):
    def __init__(self, name, price):
        super().__init__(name, price, "car")

    def rent(self):
        self.available = False

    def return_vehicle(self):
        self.available = True

    def calculate_rent(self):
        return self.price * 1.5 if self.rent_count > 3 else self.price

class Truck(Vehicle):
    def __init__(self, name, price):
        super().__init__(name, price, "truck")

    def rent(self):
        self.available = False

    def return_vehicle(self):
        self.available = True

    def calculate_rent(self):
        return self.price * 1.5 if self.rent_count > 3 else self.price

class Bike(Vehicle):
    def __init__(self, name, price):
        super().__init__(name, price, "bike")

    def rent(self):
        self.available = False

    def return_vehicle(self):
        self.available = True

    def calculate_rent(self):
        return self.price * 1.5 if self.rent_count > 3 else self.price


In [1]:
from models.vehicle import Car, Truck, Bike
from models.fleet_manager import FleetManager

def main():
    manager = FleetManager()
    manager.load_from_json()

    # Transportlar qo‘shamiz
    manager.add_vehicle(Car("Toyota", 50))
    manager.add_vehicle(Truck("Volvo Truck", 80))
    manager.add_vehicle(Bike("Mazda", 20))

    # Ijaraga olish va qaytarish
    manager.rent_vehicle("Toyota")
    manager.rent_vehicle("Mazda")
    manager.return_vehicle("Toyota")

    # Qidirish
    results = manager.search(type="truck")
    print("Qidiruv natijalari:")
    for v in results:
        print(f"{v.name} - {v.type} - ${v.calculate_rent()} - {'Available' if v.available else 'Rented'}")

    # Holatni saqlash
    manager.save_to_json()

if __name__ == "__main__":
    main()


JSON fayli topilmadi. Yangi fayl yaratiladi.
Toyota rented for $50.00
Mazda rented for $20.00
Toyota returned.
Qidiruv natijalari:
Volvo Truck - truck - $80 - Available
